# Lunchanator

## The lunch deciding utility.

Lunchanator helps groups of people decide where to eat together

Importing needed packages

In [2]:
import requests
import json
import geocoder
import numpy as np
import pandas as pd

We used the Bing geocoder because the Google limit for the month was reached via other projects.

In [3]:
g = geocoder.bing('Brooklyn, NY', key='')

Foursquare request for restaurants in the Brooklyn, NY area.

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id = '', # your Foursquare ID
  client_secret = '', # your Foursquare Secret
  v = '20180605', # Foursquare API version
  ll = str(g.json['lat']) + ',' + str(g.json['lng']),
  query = 'restaurant',
  radius = 1000,
  limit = 50,
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

Parsing results into a list containing restaurant names.

In [5]:
restaurants = []
for m in data['response']['groups'][0]['items']:
    restaurants.append(m['venue']['name'])

This block generates a pretend dataset of 20 users each having selected 15 "good" and 15 "bad" restaurants each. Note that no user can have the same restaurant in both categories.

In [6]:
d = {}
d['users'] = ['user'+str(i) for i in range(20)]
d['good restaurants'] = []
d['bad restaurants'] = []
for i in range(1,21):
    restaurant_list = np.random.choice(restaurants, size=30, replace=False)
    d['good restaurants'].append(restaurant_list[:15])
    d['bad restaurants'].append(restaurant_list[15:])

Transforming the above generated data into a dataframe.

In [7]:
user_df = pd.DataFrame.from_dict(d)

In [8]:
user_df = user_df[['users','good restaurants','bad restaurants']]

An example of user data.

In [9]:
user_df.head()

,users,good restaurants,bad restaurants
0,user0,"[Sottocasa Pizzeria, Grand Army, Luzzo's BK, D...","[Sophies Cuban Cuisine, two8two Bar & Burger, ..."
1,user1,"[Table 87, Gregory's Coffee, two8two Bar & Bur...","[Mile End Delicatessen, Five Guys, Sottocasa P..."
2,user2,"[La Bagel Delight, Bien Cuit, Chez Moi, Hibino...","[Dos Toros Taquería, Paris Baguette, French Lo..."
3,user3,"[One Girl Cookies, Chipotle Mexican Grill, Luz...","[My Little Pizzeria, DeKalb Market Hall, Sotto..."
4,user4,"[Iron Chef House, Dellarocco's, La Vara, Fast ...","[Beasts & Bottles, Maison Kayser, Yemen Cafe, ..."


Creating a group of 6 people at random that wish to have lunch together.

In [10]:
df_lunch_group = user_df.sample(n=6)

In [11]:
df_lunch_group = df_lunch_group.sort_index()

The lunch group.

In [12]:
df_lunch_group

,users,good restaurants,bad restaurants
3,user3,"[One Girl Cookies, Chipotle Mexican Grill, Luz...","[My Little Pizzeria, DeKalb Market Hall, Sotto..."
9,user9,"[Ki Sushi, Mile End Delicatessen, Grand Army, ...","[Court Street Bagels, Shelsky's of Brooklyn, M..."
11,user11,"[Five Guys, Sottocasa Pizzeria, Gregory's Coff...","[Shelsky's of Brooklyn, Doner Kebab NYC, Luzzo..."
12,user12,"[Piz-zetta, Dellarocco's, Table 87, Five Guys,...","[Dos Toros Taquería, Colonie, Luzzo's BK, Mais..."
14,user14,"[Gregory's Coffee, Yaso Tangbao, Sophies Cuban...","[Wild Ginger, Lassen & Hennigs, Piz-zetta, Don..."
15,user15,"[Sushi Gallery, River Deli, The Gumbo Bros, Ch...","[Hibino, One Girl Cookies, Bar Tabac, Henry Pu..."


Creating the list of restaurants that will be voted on.

In [13]:
lunch_votes = [(restaurant,0) for restaurant in restaurants]

In [14]:
lunch_dict = dict(lunch_votes)

Tallying votes.

In [15]:
for rest in lunch_dict.keys():
    for good_choice in df_lunch_group['good restaurants']:
        if rest in good_choice:
            lunch_dict[rest] += 1
    for bad_choice in df_lunch_group['bad restaurants']:
        if rest in bad_choice:
            lunch_dict[rest] -= 1

Conversion to properly formatted dataframe.

In [16]:
d = {'restaurants': [], 'votes': []}
for k,v in lunch_dict.items():
    d['restaurants'].append(k)
    d['votes'].append(v)

In [17]:
df_lunch_votes = pd.DataFrame.from_dict(d)

Sorting the list of restaurants by vote count.

In [18]:
df_lunch_votes_sorted = df_lunch_votes.sort_values('votes', ascending = False)

The sorted list.

In [19]:
df_lunch_votes_sorted

,restaurants,votes
8,Gregory's Coffee,4
18,Ki Sushi,4
2,La Bagel Delight,3
32,Mile End Delicatessen,3
11,French Louie,2
39,Piz-zetta,2
15,Bien Cuit,2
38,Five Guys,2
21,Hibino,2
47,Yaso Tangbao,2


A restaurant is selected at random from the top 8.

In [20]:
df_lunch_votes_sorted['restaurants'][:8].sample()

11    French Louie
Name: restaurants, dtype: object